# Creacion Consolidado Duraciones

Comenzaremos el analisis de la data consolidando la info en un unico DataSet

In [1]:
import pandas as pd
import numpy as np
import os
import warnings

from tqdm import tqdm
from unicodedata import normalize
from pjud.data import transformdata
from pjud.data import cleandata

In [2]:
warnings.filterwarnings(action="ignore")
pd.set_option('display.max_columns', 100)

In [3]:
tqdm.pandas()

In [4]:
path_processed = "../data/processed/pjud"

In [6]:
df_duracion = pd.read_feather(f"{path_processed}/processes_Duraciones.feather")
print(f"Existen : {len(df_duracion)} registros")

Existen : 3354732 registros


In [7]:
df_duracion.columns

Index(['index', 'COD. CORTE', 'CORTE', 'COD. TRIBUNAL', 'TRIBUNAL', 'RIT',
       'TIPO CAUSA', 'FECHA INGRESO', 'FECHA TERMINO', 'DURACIÓN CAUSA',
       'MOTIVO TERMINO', 'MES TERMINO', 'AÑO TERMINO', 'TOTAL TERMINOS'],
      dtype='object')

Unificar RIT y Tribunal en una sola columna para evitar mala interpretacion de causas

In [8]:
df_duracion['TRIBUNAL-RIT'] = df_duracion['COD. TRIBUNAL'].map(str) + "-" + df_duracion['RIT'].map(str)

In [9]:
df_duracion.sample(8)

,index,COD. CORTE,CORTE,COD. TRIBUNAL,TRIBUNAL,RIT,TIPO CAUSA,FECHA INGRESO,FECHA TERMINO,DURACIÓN CAUSA,MOTIVO TERMINO,MES TERMINO,AÑO TERMINO,TOTAL TERMINOS,TRIBUNAL-RIT
1359790,9819,10,C.A. DE ARICA,992,JUZGADO DE GARANTIA ARICA,8734-2016,Ordinaria,2016-11-17,2016-11-17,0,APROBACION NO INICIO INVESTIGACION.,nov-16,2016,1,992-8734-2016
235414,235582,40,C.A. DE TALCA,962,JUZGADO DE GARANTIA MOLINA,1296-2019,Ordinaria,2019-09-16,2019-09-16,0,COMUNICA Y/O APLICA DECISION PPIO. DE OPORTUNI...,sept-19,2019,1,962-1296-2019
35748,35784,15,C.A. DE ANTOFAGASTA,950,JUZGADO DE GARANTIA TOCOPILLA,92-2019,Ordinaria,2019-01-24,2019-03-11,46,SENTENCIA,mar-19,2019,1,950-92-2019
3076443,401441,56,C.A. DE PUERTO MONTT,1092,JUZGADO DE GARANTIA ANCUD,1800-2014,Ordinaria,2014-12-17,2015-02-27,72,COMUNICA Y/O APLICA DECISION PPIO. DE OPORTUNI...,feb-15,2015,1,1092-1800-2014
1911621,562044,90,C.A. DE SANTIAGO,1233,DECIMOCUARTO JUZGADO DE GARANTIA SANTIAGO,12270-2016,Ordinaria,2016-12-16,2016-12-16,0,COMUNICA Y/O APLICA DECISION PPIO. DE OPORTUNI...,dic-16,2016,1,1233-12270-2016
1998494,648994,91,C.A. DE SAN MIGUEL,1237,JUZGADO DE GARANTIA SAN BERNARDO,6001-2015,Ordinaria,2015-07-01,2016-08-25,421,DECLARA SOBRESEIMIENTO DEFINITIVO.,ago-16,2016,1,1237-6001-2015
2504151,485917,90,C.A. DE SANTIAGO,1226,SEPTIMO JUZGADO DE GARANTIA SANTIAGO,12495-2017,Ordinaria,2017-07-14,2017-07-14,0,APROBACION NO INICIO INVESTIGACION.,jul-17,2017,1,1226-12495-2017
1578433,228627,40,C.A. DE TALCA,964,JUZGADO DE GARANTIA TALCA,3629-2016,Ordinaria,2016-05-10,2016-11-04,178,NO PERSEVERAR EN EL PROCEDIMIENTO.,nov-16,2016,1,964-3629-2016


Imputo algunos datos que si dispongo del dataset.

In [10]:
df_duracion['AÑO INGRESO'] = df_duracion['RIT'].progress_apply(cleandata.obtiene_año)




100%|██████████| 3354732/3354732 [00:05<00:00, 626683.74it/s]


In [11]:
df_duracion.columns

Index(['index', 'COD. CORTE', 'CORTE', 'COD. TRIBUNAL', 'TRIBUNAL', 'RIT',
       'TIPO CAUSA', 'FECHA INGRESO', 'FECHA TERMINO', 'DURACIÓN CAUSA',
       'MOTIVO TERMINO', 'MES TERMINO', 'AÑO TERMINO', 'TOTAL TERMINOS',
       'TRIBUNAL-RIT', 'AÑO INGRESO'],
      dtype='object')

In [12]:
columnas_duplicadas = ['index']

df_duracion.drop(columnas_duplicadas, axis='columns', inplace=True)



## Carga Data relacionada a Poblacion

In [13]:
df_poblacion = pd.read_feather(f"{path_processed}/processes_DataConsolidada_Poblacion_Jurisdiccion.feather")
print(f"{len(df_poblacion)} registros")

197 registros


In [14]:
df_poblacion

,index,REGION,CORTE,TRIBUNAL,POBLACION,HOMBRES,MUJERES,URBANO,RURAL,COMUNAS,JUECES,ASIENTO,TIPO JUZGADO
0,0,REGION DE TARAPACA,C.A. DE IQUIQUE,TRIBUNAL DE JUICIO ORAL EN LO PENAL IQUIQUE,330558,167793,162765,310065,20493,"[HUARA, CAMIÑA, COLCHANE, IQUIQUE, POZO ALMONT...",13,IQUIQUE,ORAL
1,1,REGION DE ANTOFAGASTA,C.A. DE ANTOFAGASTA,TRIBUNAL DE JUICIO ORAL EN LO PENAL CALAMA,177048,92417,84631,164011,13037,"[CALAMA, OLLAGUE, SAN PEDRO DE ATACAMA]",7,CALAMA,ORAL
2,2,REGION DE ANTOFAGASTA,C.A. DE ANTOFAGASTA,TRIBUNAL DE JUICIO ORAL EN LO PENAL ANTOFAGASTA,430486,222597,207889,407737,22749,"[TOCOPILLA, MARIA ELENA, MEJILLONES, SIERRA GO...",13,ANTOFAGASTA,ORAL
3,3,REGION DE ATACAMA,C.A. DE COPIAPO,TRIBUNAL DE JUICIO ORAL EN LO PENAL COPIAPO,286168,144420,141748,260520,25648,"[CHAÑARAL, DIEGO DE ALMAGRO, CALDERA, COPIAPO,...",9,COPIAPO,ORAL
4,4,REGION DE COQUIMBO,C.A. DE LA SERENA,TRIBUNAL DE JUICIO ORAL EN LO PENAL LA SERENA,496337,239495,256842,443484,52853,"[LA HIGUERA, VICUÑA, LA SERENA, COQUIMBO, ANDA...",10,LA SERENA,ORAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,192,REGION DE AISEN DEL GENERAL CARLOS IBAÑEZ DEL ...,C.A. DE COYHAIQUE,JUZGADO DE LETRAS Y GARANTIA COCHRANE,4638,2561,2077,2841,1797,"[COCHRANE, O'HIGGINS, TORTEL]",1,COCHRANE,LETRAS Y GARANTIA
193,193,REGION DE MAGALLANES Y ANTARTICA CHILENA,C.A. DE PUNTA ARENAS,JUZGADO DE LETRAS Y GARANTIA NATALES,22686,11792,10894,19180,3506,"[NATALES, TORRES DEL PAINE]",2,NATALES,LETRAS Y GARANTIA
194,194,REGION DE MAGALLANES Y ANTARTICA CHILENA,C.A. DE PUNTA ARENAS,JUZGADO DE LETRAS Y GARANTIA PORVENIR,8364,5266,3098,6062,2302,"[PORVENIR, PRIMAVERA, TIMAUKEL]",1,PORVENIR,LETRAS Y GARANTIA
195,195,REGION DE MAGALLANES Y ANTARTICA CHILENA,C.A. DE PUNTA ARENAS,JUZGADO DE LETRAS Y GARANTIA CABO DE HORNOS,2201,1321,880,1874,327,"[CABO DE HORNOS, ANTARTICA]",1,CABO DE HORNOS,LETRAS Y GARANTIA


In [15]:
df_fulldataduracion = pd.merge(df_duracion, df_poblacion, how='left', on=['CORTE','TRIBUNAL'])

In [16]:
len(df_fulldataduracion)

3354732

In [17]:
df_fulldataduracion

,COD. CORTE,CORTE,COD. TRIBUNAL,TRIBUNAL,RIT,TIPO CAUSA,FECHA INGRESO,FECHA TERMINO,DURACIÓN CAUSA,MOTIVO TERMINO,MES TERMINO,AÑO TERMINO,TOTAL TERMINOS,TRIBUNAL-RIT,AÑO INGRESO,index,REGION,POBLACION,HOMBRES,MUJERES,URBANO,RURAL,COMUNAS,JUECES,ASIENTO,TIPO JUZGADO
0,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,100-2019,Ordinaria,2019-04-03,2019-05-17,44,SENTENCIA,may-19,2019,1,988-100-2019,2019,29,REGION DE ARICA Y PARINACOTA,226068,112581,113487,207231,18837,"[GENERAL LAGOS, PUTRE, ARICA, CAMARONES]",10,ARICA,ORAL
1,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,101-2019,Ordinaria,2019-04-04,2019-05-17,43,SENTENCIA,may-19,2019,1,988-101-2019,2019,29,REGION DE ARICA Y PARINACOTA,226068,112581,113487,207231,18837,"[GENERAL LAGOS, PUTRE, ARICA, CAMARONES]",10,ARICA,ORAL
2,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,10-2019,Ordinaria,2019-01-04,2019-01-29,25,SENTENCIA,ene-19,2019,1,988-10-2019,2019,29,REGION DE ARICA Y PARINACOTA,226068,112581,113487,207231,18837,"[GENERAL LAGOS, PUTRE, ARICA, CAMARONES]",10,ARICA,ORAL
3,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,102-2019,Ordinaria,2019-04-04,2019-11-20,230,SENTENCIA,nov-19,2019,1,988-102-2019,2019,29,REGION DE ARICA Y PARINACOTA,226068,112581,113487,207231,18837,"[GENERAL LAGOS, PUTRE, ARICA, CAMARONES]",10,ARICA,ORAL
4,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,103-2019,Ordinaria,2019-04-05,2019-04-10,5,DECLARA SOBRESEIMIENTO DEFINITIVO.,abr-19,2019,1,988-103-2019,2019,29,REGION DE ARICA Y PARINACOTA,226068,112581,113487,207231,18837,"[GENERAL LAGOS, PUTRE, ARICA, CAMARONES]",10,ARICA,ORAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3354727,91,C.A. DE SAN MIGUEL,1355,TRIBUNAL DE JUICIO ORAL EN LO PENAL MELIPILLA,86-2014,Ordinaria,2014-12-02,2015-01-31,60,SENTENCIA,ene-15,2015,1,1355-86-2014,2014,33,REGION METROPOLITANA DE SANTIAGO,185966,93110,92856,114535,71431,"[MELIPILLA, SAN PEDRO, ALHUE, CURACAVI, MARIA ...",6,MELIPILLA,ORAL
3354728,91,C.A. DE SAN MIGUEL,1355,TRIBUNAL DE JUICIO ORAL EN LO PENAL MELIPILLA,87-2014,Ordinaria,2014-12-09,2015-01-28,50,SENTENCIA,ene-15,2015,1,1355-87-2014,2014,33,REGION METROPOLITANA DE SANTIAGO,185966,93110,92856,114535,71431,"[MELIPILLA, SAN PEDRO, ALHUE, CURACAVI, MARIA ...",6,MELIPILLA,ORAL
3354729,91,C.A. DE SAN MIGUEL,1355,TRIBUNAL DE JUICIO ORAL EN LO PENAL MELIPILLA,88-2014,Ordinaria,2014-12-30,2015-02-02,34,SENTENCIA,feb-15,2015,1,1355-88-2014,2014,33,REGION METROPOLITANA DE SANTIAGO,185966,93110,92856,114535,71431,"[MELIPILLA, SAN PEDRO, ALHUE, CURACAVI, MARIA ...",6,MELIPILLA,ORAL
3354730,91,C.A. DE SAN MIGUEL,1355,TRIBUNAL DE JUICIO ORAL EN LO PENAL MELIPILLA,90-2013,Ordinaria,2013-12-18,2015-08-17,607,SENTENCIA,ago-15,2015,1,1355-90-2013,2013,33,REGION METROPOLITANA DE SANTIAGO,185966,93110,92856,114535,71431,"[MELIPILLA, SAN PEDRO, ALHUE, CURACAVI, MARIA ...",6,MELIPILLA,ORAL


In [18]:
df_fulldataduracion.columns

Index(['COD. CORTE', 'CORTE', 'COD. TRIBUNAL', 'TRIBUNAL', 'RIT', 'TIPO CAUSA',
       'FECHA INGRESO', 'FECHA TERMINO', 'DURACIÓN CAUSA', 'MOTIVO TERMINO',
       'MES TERMINO', 'AÑO TERMINO', 'TOTAL TERMINOS', 'TRIBUNAL-RIT',
       'AÑO INGRESO', 'index', 'REGION', 'POBLACION', 'HOMBRES', 'MUJERES',
       'URBANO', 'RURAL', 'COMUNAS', 'JUECES', 'ASIENTO', 'TIPO JUZGADO'],
      dtype='object')

VERIFICAR FILAS A ELIMINAR ... CAMBIARON ....

In [19]:
columnas_duplicadas = ['index']

df_fulldataduracion.drop(columnas_duplicadas, axis='columns', inplace=True)


In [20]:
#df_consolidado.drop(['MATERIA_x','FECHA TERMINO_x','AÑO TERMINO_x','TIPO CAUSA_y'], axis='columns', inplace=True)

In [21]:
df_fulldataduracion.sample(5)

,COD. CORTE,CORTE,COD. TRIBUNAL,TRIBUNAL,RIT,TIPO CAUSA,FECHA INGRESO,FECHA TERMINO,DURACIÓN CAUSA,MOTIVO TERMINO,MES TERMINO,AÑO TERMINO,TOTAL TERMINOS,TRIBUNAL-RIT,AÑO INGRESO,REGION,POBLACION,HOMBRES,MUJERES,URBANO,RURAL,COMUNAS,JUECES,ASIENTO,TIPO JUZGADO
1120963,90,C.A. DE SANTIAGO,1221,SEGUNDO JUZGADO DE GARANTIA SANTIAGO,14238-2018,Ordinaria,2018-12-07,2018-12-07,0,APROBACION NO INICIO INVESTIGACION.,dic-18,2018,1,1221-14238-2018,2018,REGION METROPOLITANA DE SANTIAGO,583187,286136,297051,582536,651,"[QUILICURA, HUECHURABA, RENCA, CONCHALI]",10,SANTIAGO,GARANTIA
1763062,61,C.A. DE PUNTA ARENAS,995,JUZGADO DE GARANTIA PUNTA ARENAS,1905-2016,Ordinaria,2016-07-09,2016-08-04,26,DECLARA SOBRESEIMIENTO DEFINITIVO.,ago-16,2016,1,995-1905-2016,2016,DUOREGION DE MAGALLANES Y LA ANTARTICA CHILENA,133282,66870,66412,125932,7350,"[LAGUNA BLANCA, SAN GREGORIO, RIO VERDE, PUNTA...",4,PUNTA ARENAS,GARANTIA
1111216,90,C.A. DE SANTIAGO,1220,PRIMER JUZGADO DE GARANTIA SANTIAGO,3519-2018,Ordinaria,2018-06-26,2018-06-26,0,APROBACION NO INICIO INVESTIGACION.,jun-18,2018,1,1220-3519-2018,2018,REGION METROPOLITANA DE SANTIAGO,230293,112412,117881,226138,4155,[PUDAHUEL],5,SANTIAGO,GARANTIA
2298133,46,C.A. DE CONCEPCION,1077,JUZGADO DE GARANTIA ARAUCO,508-2016,Ordinaria,2016-06-01,2017-04-24,327,SENTENCIA,abr-17,2017,1,1077-508-2016,2016,REGION DEL BIO BIO,36257,17875,18382,27320,8937,[ARAUCO],1,ARAUCO,GARANTIA
3264255,90,C.A. DE SANTIAGO,1235,JUZGADO DE GARANTIA COLINA,5618-2015,Ordinaria,2015-12-21,2015-12-21,0,ACOGE REQUERIMIENTO (MONITORIO).,dic-15,2015,1,1235-5618-2015,2015,REGION METROPOLITANA DE SANTIAGO,267553,136227,131326,211849,55704,"[TIL TIL, COLINA, LAMPA]",4,COLINA,GARANTIA


Reordenando Nombres de las columnas ...

In [23]:
df_fulldataduracion.rename(columns = {'COD. CORTE':'cod_corte',
                                        'COD. TRIBUNAL':'cod_tribunal',
                                         'RIT':'rit',
                                         'CORTE':'corte',
                                         'TRIBUNAL':'tribunal',
                                         'TIPO CAUSA':'tipo_causa',
                                         'FECHA INGRESO':'fecha_ingreso',
                                         'AÑO INGRESO':'año_ingreso',
                                         'FECHA TERMINO':'fecha_termino',
                                         'MES TERMINO':'mes_termino',
                                         'AÑO TERMINO':'año_termino',
                                         'TOTAL TERMINOS':'total_terminos',
                                         'DURACIÓN CAUSA':'duracion_causa',
                                         'TRIBUNAL-RIT':'tribunal_rit',
                                         'MOTIVO TERMINO':'motivo_termino',
                                         'REGION':'region',
                                         'POBLACION':'poblacion',
                                         'HOMBRES':'hombres',
                                         'MUJERES':'mujeres',
                                         'URBANO':'urbano',
                                         'RURAL':'rural',
                                         'COMUNAS':'comunas',
                                         'JUECES':'dotacion_jueces',
                                         'ASIENTO':'asiento',
                                         'TIPO JUZGADO':'tipo_juzgado'
                                     },inplace = True)

In [24]:
df_fulldataduracion.columns

Index(['cod_corte', 'corte', 'cod_tribunal', 'tribunal', 'rit', 'tipo_causa',
       'fecha_ingreso', 'fecha_termino', 'duracion_causa', 'motivo_termino',
       'mes_termino', 'año_termino', 'total_terminos', 'tribunal_rit',
       'año_ingreso', 'region', 'poblacion', 'hombres', 'mujeres', 'urbano',
       'rural', 'comunas', 'dotacion_jueces', 'asiento', 'tipo_juzgado'],
      dtype='object')

In [26]:
df_fulldataduracion = df_fulldataduracion[['region','cod_corte','corte','tribunal_rit','cod_tribunal','rit','tribunal','tipo_juzgado','dotacion_jueces','tipo_causa',
                                           'año_ingreso','fecha_ingreso','año_termino','mes_termino','fecha_termino','motivo_termino','total_terminos','duracion_causa',
                                           'asiento','comunas','poblacion','hombres','mujeres','urbano','rural']]

In [27]:
df_fulldataduracion.sample(5)

,region,cod_corte,corte,tribunal_rit,cod_tribunal,rit,tribunal,tipo_juzgado,dotacion_jueces,tipo_causa,año_ingreso,fecha_ingreso,año_termino,mes_termino,fecha_termino,motivo_termino,total_terminos,duracion_causa,asiento,comunas,poblacion,hombres,mujeres,urbano,rural
964640,REGION DEL BIO BIO,46,C.A. DE CONCEPCION,1076-138-2008,1076,138-2008,JUZGADO DE GARANTIA CAÑETE,GARANTIA,1,Ordinaria,2008,2008-02-22,2018,jul-18,2018-07-11,DECLARA SOBRESEIMIENTO DEFINITIVO.,1,3792,CAÑETE,"[CAÑETE, CONTULMO, TIRUA]",50985,24864,26121,26293,24692
2793889,REGION DE VALPARAISO,30,C.A. DE VALPARAISO,1046-3551-2015,1046,3551-2015,JUZGADO DE GARANTIA VIÑA DEL MAR,GARANTIA,8,Ordinaria,2015,2015-03-30,2015,mar-15,2015-03-31,COMUNICA Y/O APLICA DECISION PPIO. DE OPORTUNI...,1,1,VIÑA DEL MAR,"[VIÑA DEL MAR, CONCON]",376400,178990,197410,373657,2743
1043783,REGION DE LA ARAUCANIA,50,C.A. DE TEMUCO,944-661-2018,944,661-2018,JUZGADO DE GARANTIA LONCOCHE,GARANTIA,1,Ordinaria,2018,2018-09-18,2018,dic-18,2018-12-17,SENTENCIA,1,90,LONCOCHE,[LONCOCHE],23612,11502,12110,16592,7020
1925300,REGION METROPOLITANA DE SANTIAGO,90,C.A. DE SANTIAGO,1233-9416-2015,1233,9416-2015,DECIMOCUARTO JUZGADO DE GARANTIA SANTIAGO,GARANTIA,9,Ordinaria,2015,2015-10-13,2016,nov-16,2016-11-25,DECLARA SOBRESEIMIENTO DEFINITIVO.,1,409,SANTIAGO,[LA FLORIDA],366916,175693,191223,366799,117
2756509,REGION DE COQUIMBO,25,C.A. DE LA SERENA,929-4357-2012,929,4357-2012,JUZGADO DE GARANTIA LA SERENA,GARANTIA,3,Ordinaria,2012,2012-10-05,2015,feb-15,2015-02-19,DECLARA SOBRESEIMIENTO DEFINITIVO.,1,867,LA SERENA,"[LA SERENA, LA HIGUERA]",225295,108072,117223,201933,23362


In [28]:
# Directorio donde se guardaran archivos feather
df_fulldataduracion.reset_index(inplace = True)
os.makedirs(path_processed, exist_ok = True)  

# Guardamos dataset como archivo feather

df_fulldataduracion.to_feather(f'{path_processed}/consolidated_FULLDATA_DURACION.feather')